<a href="https://colab.research.google.com/github/FIIT-Bc-projects/2022-Petrik-AbstractiveSummarization/blob/main/research/cnn_prepr_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN daily mail processing testing

In [1]:
# !pip install datasets

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
from datasets import load_dataset
from time import time
import spacy
import re
import spacy

spacy.prefer_gpu() # IMPORTANT= enable colab gpu for spacy
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])

print("Libs loaded.")

Libs loaded.


## CNN dataset load

In [3]:
dataset = load_dataset("cnn_dailymail", "3.0.0")

  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [5]:
def base_tidy(df):
  df.drop('id', axis=1, inplace=True)
  df.rename(columns={'article': 'text', 'highlights': 'summary'}, inplace=True)

ds_train = dataset['train'].to_pandas()
ds_test = dataset['test'].to_pandas()
ds_val = dataset['validation'].to_pandas()

base_tidy(ds_train)
base_tidy(ds_test)
base_tidy(ds_val)


ds_train.shape, ds_test.shape, ds_val.shape

((287113, 2), (11490, 2), (13368, 2))

In [6]:
ds_train

,text,summary
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa..."
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non..."
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical..."
...,...,...
287108,"The nine-year-old daughter of a black, unarmed...","Rumain Brisbon, 34, was killed after Phoenix p..."
287109,Legalising assisted suicide is a slippery slop...,"Theo Boer, a European assisted suicide watchdo..."
287110,A group calling itself 'The Women of the 99 Pe...,Ohio congressman criticised for 'condoning the...
287111,Most men enjoy a good pint of lager or real al...,The Black Country Ale Tairsters have been to 1...


In [7]:
ds_test

,text,summary
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b..."
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...
...,...,...
11485,Telecom watchdogs are to stop a rip-off that a...,Operators are charging up to 20p a minute - ev...
11486,The chilling reenactment of how executions are...,Bali Nine ringleaders will face the firing squ...
11487,It is a week which has seen him in deep water ...,Hardy was convicted of domestic abuse against ...
11488,"Despite the hype surrounding its first watch, ...",Apple sold more than 61 million iPhones in the...


In [8]:
ds_val

,text,summary
0,"(CNN)Share, and your gift will be multiplied. ...",Zully Broussard decided to give a kidney to a ...
1,"(CNN)On the 6th of April 1996, San Jose Clash ...",The 20th MLS season begins this weekend .\nLea...
2,"(CNN)French striker Bafetimbi Gomis, who has a...",Bafetimbi Gomis collapses within 10 minutes of...
3,(CNN)It was an act of frustration perhaps more...,Rory McIlroy throws club into water at WGC Cad...
4,(CNN)A Pennsylvania community is pulling toget...,"Cayman Naib, 13, hasn't been heard from since ..."
...,...,...
13363,"It is the dream of many young children, the ch...","The town in Valley Center, San Diego, has been..."
13364,It’s the type of encounter that can send panic...,Photographer Graham Hewer captured the jaw-dro...
13365,A group of tourists to the Bahamas enjoyed one...,The pigs swim through the crystal clear sea an...
13366,Pippa Middleton bundled up against the London ...,Kate's sister is back in London following 'cri...


In [9]:
example = ds_train.iloc[0]
print('Text: ', example.text)
print('Summary: ', example.summary)

Text:  LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details

## Preprocessing testing

In [12]:

# https://www.kaggle.com/code/sandeepbhogaraju/text-summarization-with-seq2seq-model/notebook
#Removes non-alphabetic characters:
def text_strip(column):
    for row in column:
        
        #ORDER OF REGEX IS VERY VERY IMPORTANT!!!!!!
        
        row=re.sub("(\\t)", ' ', str(row)).lower() #remove escape charecters
        row=re.sub("(\\r)", ' ', str(row)).lower() 
        row=re.sub("(\\n)", ' ', str(row)).lower()
        
        row=re.sub("(__+)", ' ', str(row)).lower()   #remove _ if it occors more than one time consecutively
        row=re.sub("(--+)", ' ', str(row)).lower()   #remove - if it occors more than one time consecutively
        row=re.sub("(~~+)", ' ', str(row)).lower()   #remove ~ if it occors more than one time consecutively
        row=re.sub("(\+\++)", ' ', str(row)).lower()   #remove + if it occors more than one time consecutively
        row=re.sub("(\.\.+)", ' ', str(row)).lower()   #remove . if it occors more than one time consecutively
        
        row=re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]", ' ', str(row)).lower() #remove <>()|&©ø"',;?~*!
        
        row=re.sub("(mailto:)", ' ', str(row)).lower() #remove mailto:
        row=re.sub(r"(\\x9\d)", ' ', str(row)).lower() #remove \x9* in text
        row=re.sub("([iI][nN][cC]\d+)", 'INC_NUM', str(row)).lower() #replace INC nums to INC_NUM
        row=re.sub("([cC][mM]\d+)|([cC][hH][gG]\d+)", 'CM_NUM', str(row)).lower() #replace CM# and CHG# to CM_NUM
        
        
        row=re.sub("(\.\s+)", ' ', str(row)).lower() #remove full stop at end of words(not between)
        row=re.sub("(\-\s+)", ' ', str(row)).lower() #remove - at end of words(not between)
        row=re.sub("(\:\s+)", ' ', str(row)).lower() #remove : at end of words(not between)
        
        row=re.sub("(\s+.\s+)", ' ', str(row)).lower() #remove any single charecters hanging between 2 spaces
        
        #Replace any url as such https://abc.xyz.net/browse/sdf-5327 ====> abc.xyz.net
        try:
            url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', str(row))
            repl_url = url.group(3)
            row = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)',repl_url, str(row))
        except:
            pass #there might be emails with no url in them
        

        
        row = re.sub("(\s+)",' ',str(row)).lower() #remove multiple spaces
        
        #Should always be last
        row=re.sub("(\s+.\s+)", ' ', str(row)).lower() #remove any single charecters hanging between 2 spaces
        yield row

clean_testds_text = text_strip(ds_test['text'])



t = time()
text = [str(doc).strip() for doc in nlp.pipe(clean_testds_text)]
print('Test ds cleanup - text: {} mins'.format(round((time() - t) / 60, 2)))

Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 398, in _send_bytes
    self._send(buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


Time to clean up everything: 3.44 mins


In [16]:
text[0]

'cnn the palestinian authority officially became the 123rd member of the international criminal court on wednesday step that gives the court jurisdiction over alleged crimes in palestinian territories the formal accession was marked with ceremony at the hague in the netherlands where the court is based the palestinians signed the icc founding rome statute in january when they also accepted its jurisdiction over alleged crimes committed in the occupied palestinian territory including east jerusalem since june 13 2014 later that month the icc opened preliminary examination into the situation in palestinian territories paving the way for possible war crimes investigations against israelis as members of the court palestinians may be subject to counter-charges as well israel and the united states neither of which is an icc member opposed the palestinians efforts to join the body but palestinian foreign minister riad al-malki speaking at wednesday ceremony said it was move toward greater jus

In [17]:
clean_testds_summary = text_strip(ds_test['summary'])
t = time()
summary = [str(doc).strip() for doc in nlp.pipe(clean_testds_summary)]
print('Test ds cleanup - summary: {} mins'.format(round((time() - t) / 60, 2)))
summary[0]

Test ds cleanup - summary: 0.28 mins


'membership gives the icc jurisdiction over alleged crimes committed in palestinian territories since last june israel and the united states opposed the move which could open the door to war crimes investigations against israelis .'

**yaaay**